<a href="https://colab.research.google.com/github/rvkushnir/project_fifa_players/blob/main/notebooks/02_analytical_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ініціалізація середовища**

Клонуємо репозиторій з GitHub у /content, переходимо в корінь проєкту.
Це гарантує однакові шляхи для даних/виводів у всіх сесіях Colab.
Після клонування перевіряємо наявність data/raw/fifa_players.csv.
Вихід: робочий каталог → pwd = /content/project_fifa_players.

In [1]:
#  Ініціалізація середовища: клон репозиторію і перехід у робочу теку
REPO = "rvkushnir/project_fifa_players"
BRANCH = "main"
WORKDIR = "/content/project_fifa_players"

# чистий клон
!rm -rf "$WORKDIR"
!git clone --depth 1 -b "$BRANCH" "https://github.com/{REPO}.git" "$WORKDIR"
%cd "$WORKDIR"

# Якщо у репо використовується Git LFS (великі файли) — підтягнемо дані
!git lfs install 2>/dev/null || true
!git lfs pull 2>/dev/null || true

# Перевіримо, що CSV на місці
!ls -lah data/raw


Cloning into '/content/project_fifa_players'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 2.74 MiB | 9.82 MiB/s, done.
/content/project_fifa_players
Updated git hooks.
Git LFS initialized.
total 11M
drwxr-xr-x 2 root root 4.0K Sep 12 19:58 .
drwxr-xr-x 4 root root 4.0K Sep 12 19:58 ..
-rw-r--r-- 1 root root  11M Sep 12 19:58 fifa_players.csv
-rw-r--r-- 1 root root   61 Sep 12 19:58 README.md


Мета блоку.
Побудувати узгоджений набір аналітичних таблиць для пунктів (1–7):

1) топ-100 гравців за рейтингом;
2) топ-100 за зарплатою та їх порівняння;
3) топ-30 воротарів;
4) топ-30 клубів за середнім рейтингом;
5) топ-30 клубів за середньою швидкістю;
6) ліги з найкращим дриблінгом;
7) рейтинг клубів за “стартовим XI” (1–4–4–2).

Вхід.

data/interim/players_clean.parquet (кеш із Блоку 1) або data/raw/fifa_players.csv (якщо кеш відсутній).

Вихід (CSV → out/tables/).

top100_overall.csv, top100_wage.csv, compare_top100_summary.csv,
top100_only_overall.csv, top100_only_wage.csv, top100_intersection.csv

top30_goalkeepers_overall.csv

clubs_avg_overall_all.csv, clubs_avg_overall_top30_min11.csv

clubs_avg_speed_all.csv, clubs_avg_speed_top30_min11.csv

leagues_avg_dribbling_all.csv, leagues_avg_dribbling_min200.csv

clubs_best_starting_xi_top30.csv

Ключові правила / параметри.

Сортування стабілізоване тай-брейками:

за рейтингом: overall ↓, potential ↓, wage_eur ↓, age ↑;

за зарплатою: wage_eur ↓, overall ↓, age ↑.

Швидкість: базово movement_sprint_speed (якщо немає — pace).

Дриблінг по лігах: зберігаємо всі та версію з порогом вибірки MIN_LEAGUE_PLAYERS=200.

“Стартовий XI” (1–4–4–2):

Гнучкий режим можливий: FLEXIBLE_POSITIONS=True — гравець може займати будь-яку зі своїх позицій (задача призначення), штраф за не-primary роль NON_PRIMARY_PENALTY=2.0.

Жорсткий режим: FLEXIBLE_POSITIONS=False — беремо лише primary_position.

Якщо клуб не має достатньо валідних гравців по лініях — він пропускається.

План аналітичного блоку.
1. Побудуємо ТОП-100 за overall.
2. Побудуємо ТОП-100 за wage_uer, порівняємо з 1: перетин, розбіжності, Jaccard.
3. ТОП-30 воротарів за overall.
4. ТОП-30 клубів за середнім overall.
5. ТОП-30 клубів за середньою швидкістю (використаємо movement_sprint_speed, або pace як fallback).
6. Ліги з найвищим середнім dribbling (+ розмір вибірки).
7. Рейтинг клубів за стартовим складом 11 гравців (1 GK, 4  DEF, 4 MID, 2 FWD) на основі primary_position.


In [6]:
#  Завантаження даних (CSV)
from pathlib import Path
import pandas as pd
import numpy as np

DATA_CSV = Path("data/raw/fifa_players.csv")
OUT = Path("out/tables"); OUT.mkdir(parents=True, exist_ok=True)

def make_primary_position(s: pd.Series) -> pd.Series:
    # Беремо перший запис з player_positions як primary
    return s.astype(str).str.split(",", n=1, expand=True)[0].str.strip()

# читаємо CSV і готуємо df
if not DATA_CSV.exists():
    raise FileNotFoundError(f"Не знайдено файл: {DATA_CSV.resolve()}")

df = pd.read_csv(DATA_CSV, low_memory=False)
if "player_positions" not in df.columns:
    raise ValueError("Не знайдено стовпця player_positions.")
df["primary_position"] = make_primary_position(df["player_positions"])
for col in ["overall","potential","wage_eur","age"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# сформувати club_df для групувань
club_df = df.dropna(subset=["club_name"]) if "club_name" in df.columns else df.copy()

#  Базові колонки для зручності виводу
base_cols = [c for c in [
    "sofifa_id","short_name","long_name","age","overall","potential",
    "value_eur","wage_eur","club_name","league_name","nationality_name","primary_position"
] if c in df.columns]

def _stable_sort(_df, cols, asc):
    cols2 = [c for c in cols if c in _df.columns]
    asc2  = [asc[i] for i,c in enumerate(cols) if c in _df.columns]
    return _df.sort_values(by=cols2, ascending=asc2) if cols2 else _df

# (1) Топ-100 за рейтингом
top100_overall = (
    _stable_sort(df, ["overall","potential","wage_eur","age"], [False, False, False, True])
      .loc[:, base_cols].head(100)
)
top100_overall.to_csv(OUT/"top100_overall.csv", index=False)

# (2) Топ-100 за зарплатою + порівняння з (1)
top100_wage = (
    _stable_sort(df, ["wage_eur","overall","age"], [False, False, True])
      .loc[:, base_cols].head(100)
)
top100_wage.to_csv(OUT/"top100_wage.csv", index=False)

set_overall = set(top100_overall["sofifa_id"])
set_wage = set(top100_wage["sofifa_id"])
intersection_ids = set_overall & set_wage
only_overall_ids = set_overall - set_wage
only_wage_ids    = set_wage - set_overall
jaccard = len(intersection_ids) / len(set_overall | set_wage) if (set_overall | set_wage) else 0.0

pd.DataFrame([
    {"metric":"count_top100_overall", "value": len(set_overall)},
    {"metric":"count_top100_wage",    "value": len(set_wage)},
    {"metric":"intersection",         "value": len(intersection_ids)},
    {"metric":"jaccard_index",        "value": round(jaccard, 4)}
]).to_csv(OUT/"compare_top100_summary.csv", index=False)

df[df["sofifa_id"].isin(only_overall_ids)][base_cols].to_csv(OUT/"top100_only_overall.csv", index=False)
df[df["sofifa_id"].isin(only_wage_ids)][base_cols].to_csv(OUT/"top100_only_wage.csv", index=False)
df[df["sofifa_id"].isin(intersection_ids)][base_cols].to_csv(OUT/"top100_intersection.csv", index=False)

# (3) Топ-30 воротарів за рейтингом
gk = df[df["primary_position"]=="GK"].copy()
top30_gk = _stable_sort(gk, ["overall","potential","age"], [False, False, True]).loc[:, base_cols].head(30)
top30_gk.to_csv(OUT/"top30_goalkeepers_overall.csv", index=False)

# (4) Топ-30 клубів за середнім рейтингом (з/без порогу 11 гравців)
club_agg = (
    club_df.groupby("club_name", as_index=False)
      .agg(avg_overall=("overall","mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_overall","n_players"], ascending=[False, False])
)
club_agg.to_csv(OUT/"clubs_avg_overall_all.csv", index=False)
club_agg[club_agg["n_players"]>=11].head(30).to_csv(OUT/"clubs_avg_overall_top30_min11.csv", index=False)

# (5) Топ-30 клубів за середньою швидкістю
speed_col = "movement_sprint_speed" if "movement_sprint_speed" in df.columns else ("pace" if "pace" in df.columns else None)
if speed_col is None:
    raise ValueError("Не знайдено стовпців швидкості (movement_sprint_speed чи pace).")
club_speed = (
    club_df.groupby("club_name", as_index=False)
      .agg(avg_speed=(speed_col,"mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_speed","n_players"], ascending=[False, False])
)
club_speed.to_csv(OUT/"clubs_avg_speed_all.csv", index=False)
club_speed[club_speed["n_players"]>=11].head(30).to_csv(OUT/"clubs_avg_speed_top30_min11.csv", index=False)

# (6) Ліги з найкращим дриблінгом
if "dribbling" not in df.columns:
    raise ValueError("Не знайдено стовпця dribbling.")
league_drib = (
    df.groupby("league_name", as_index=False)
      .agg(avg_dribbling=("dribbling","mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_dribbling","n_players"], ascending=[False, False])
)
league_drib.to_csv(OUT/"leagues_avg_dribbling_all.csv", index=False)

# (7) Клубний рейтинг за 'стартовим складом 11'
from scipy.optimize import linear_sum_assignment

DEF = {"CB", "LB", "RB", "LWB", "RWB"}
MID = {"CDM", "CM", "CAM", "LM", "RM"}
FWD = {"ST", "CF", "LW", "RW"}

FLEXIBLE_POSITIONS = True      # увімк/вимк гнучкий підбір
NON_PRIMARY_PENALTY = 2.0      # штраф за гру не на основній ролі (бали overall)

def _group_of(pos: str) -> str:
    # Мапимо позицію до групи
    if pos == "GK":
        return "GK"
    if pos in DEF:
        return "DEF"
    if pos in MID:
        return "MID"
    if pos in FWD:
        return "FWD"
    return "OTHER"

def _eligible_for_group(positions_set: set, group_name: str) -> bool:
    # Перевіряємо придатність за будь-якою позицією у списку
    if not isinstance(positions_set, (set, frozenset)):
        return False
    if group_name == "GK":
        return "GK" in positions_set
    if group_name == "DEF":
        return len(positions_set & DEF) > 0
    if group_name == "MID":
        return len(positions_set & MID) > 0
    if group_name == "FWD":
        return len(positions_set & FWD) > 0
    return False

def _to_positions_set(v):
    # перетворення у множину позицій з безпекою для NaN/порожніх
    if isinstance(v, (set, frozenset)):
        return set(v)
    if isinstance(v, (list, tuple)):
        return {str(x).strip() for x in v if pd.notna(x) and str(x).strip()}
    if pd.isna(v):
        return set()
    s = str(v).strip()
    if not s or s.lower() == "nan":
        return set()
    return {x.strip() for x in s.split(",") if x.strip()}

def _rigid_xi(sub: pd.DataFrame):
    # жорсткий відбір суто за primary_position
    s = sub.sort_values(["overall", "potential"], ascending=[False, False]).drop_duplicates("sofifa_id")
    gk  = s[s["primary_position"] == "GK"].head(1)
    de  = s[s["primary_position"].isin(DEF)].head(4)
    md  = s[s["primary_position"].isin(MID)].head(4)
    fw  = s[s["primary_position"].isin(FWD)].head(2)
    if (len(gk) < 1) or (len(de) < 4) or (len(md) < 4) or (len(fw) < 2):
        return None
    xi = pd.concat([gk, de, md, fw], axis=0)
    return {
        "avg_overall_xi": xi["overall"].mean(),
        "players_ids": list(xi["sofifa_id"].values),
        "n_selected": len(xi),
    }

def best_starting_xi_score(club_df: pd.DataFrame):
    # Дві поведінки: гнучко за списком позицій + fallback на жорсткий; або одразу жорстко
    sub = club_df.copy().drop_duplicates("sofifa_id")

    if FLEXIBLE_POSITIONS and "player_positions" in sub.columns:
        # Гнучкий підбір через Hungarian
        slots = (["GK"] * 1) + (["DEF"] * 4) + (["MID"] * 4) + (["FWD"] * 2)
        n_slots = len(slots)

        sub2 = sub[pd.notna(sub["overall"])]
        if not sub2.empty:
            players = sub2.reset_index(drop=True)
            players["positions_set"] = sub2["player_positions"].apply(_to_positions_set)
            players["positions_set"] = players["positions_set"].apply(_to_positions_set)  # нормалізація
            players["primary_group"] = sub2["primary_position"].astype(str).apply(_group_of)

            # Перевірка достатності кандидатів за групами
            need = {"GK": 1, "DEF": 4, "MID": 4, "FWD": 2}
            has_gk  = (players["positions_set"].apply(lambda s: isinstance(s, (set, frozenset)) and "GK" in s)).sum()
            has_def = (players["positions_set"].apply(lambda s: isinstance(s, (set, frozenset)) and len(s & DEF) > 0)).sum()
            has_mid = (players["positions_set"].apply(lambda s: isinstance(s, (set, frozenset)) and len(s & MID) > 0)).sum()
            has_fwd = (players["positions_set"].apply(lambda s: isinstance(s, (set, frozenset)) and len(s & FWD) > 0)).sum()
            have = {"GK": int(has_gk), "DEF": int(has_def), "MID": int(has_mid), "FWD": int(has_fwd)}

            if not any(have[g] < need[g] for g in need):
                # Матриця оцінок: overall - штраф, якщо не первинна група
                scores = np.full((len(players), n_slots), -1e9, dtype=float)
                for i in range(len(players)):
                    ov = float(players.loc[i, "overall"])
                    pset = players.loc[i, "positions_set"]
                    pgrp = players.loc[i, "primary_group"]
                    for j, grp in enumerate(slots):
                        if _eligible_for_group(pset, grp):
                            penalty = 0.0 if pgrp == grp else NON_PRIMARY_PENALTY
                            scores[i, j] = ov - penalty

                cost = -scores
                row_ind, col_ind = linear_sum_assignment(cost)
                chosen = [(r, c) for r, c in zip(row_ind, col_ind) if scores[r, c] > -1e8]
                if len(chosen) >= n_slots:
                    sel_idx = [r for r, _ in sorted(chosen, key=lambda x: x[1])]
                    xi = players.iloc[sel_idx]
                    return {
                        "avg_overall_xi": xi["overall"].mean(),
                        "players_ids": list(xi["sofifa_id"].values),
                        "n_selected": len(xi),
                    }
        # fallback на жорсткий
        return _rigid_xi(sub)

    # одразу жорсткий режим
    return _rigid_xi(sub)

rows = []
for club, grp in club_df.groupby("club_name"):
    res = best_starting_xi_score(grp)
    if res:
        rows.append({"club_name": club, **res})

if rows:
    xi_table = pd.DataFrame(rows).sort_values("avg_overall_xi", ascending=False)
    xi_table.head(30).to_csv(OUT / "clubs_best_starting_xi_top30.csv", index=False)
else:
    # безпечне збереження порожньої таблиці
    pd.DataFrame(columns=["club_name", "avg_overall_xi", "players_ids", "n_selected"]).to_csv(
        OUT / "clubs_best_starting_xi_top30.csv", index=False
    )

print(f"[INFO] Клубів з валідним XI: {len(rows)}")
print("OK. Результати (1–7) збережено у out/tables/")




[INFO] Клубів з валідним XI: 691
OK. Результати (1–7) збережено у out/tables/
